In [7]:
import requests
import json
import csv
from datetime import datetime
from pandas import json_normalize

# Términos de búsqueda que deseas analizar
search_terms = ["Huawei", "Motorola", "Apple"]

# Inicializa una lista para almacenar los IDs recopilados
item_ids = []

# Recorre cada término de búsqueda
for term in search_terms:
    url = f"https://api.mercadolibre.com/sites/MLA/search?q={term}&limit=50"
    response = requests.get(url)
    data = response.json()

    # Recopila los IDs de los productos
    for item in data.get('results', []):
        item_ids.append(item['id'])


In [8]:
# Inicializa una lista para almacenar los datos JSON
json_data_list = []

# Realiza solicitudes GET por Item_Id y muestra la información
for item_id in item_ids:
    url = f"https://api.mercadolibre.com/items/{item_id}"
    response = requests.get(url)
    item_data = response.json()

    # Buscar el atributo "BRAND" dentro de la lista de atributos
    brand = None
    for attribute in item_data["attributes"]:
        if attribute["id"] == "BRAND":
            brand = attribute["value_name"]
            break
    # Buscar el atributo "MODEL" dentro de la lista de atributos
    model = None
    for attribute in item_data["attributes"]:
        if attribute["id"] == "MODEL":
            model = attribute["value_name"]
            break

    # Obtén la fecha en formato "dd-mm-yyyy"
    date_created = datetime.strptime(item_data['date_created'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%d-%m-%Y')

 # Agrega los datos a la lista
    json_data_list.append({
    'Marca': brand,
    'Modelo': model,
    'Tipo producto': item_data['domain_id'].split('-')[-1].capitalize(),
    'Condicion': item_data['condition'],
    'Status': item_data['status'],
    'Precio': item_data['price'],
    'Moneda': item_data['currency_id'],
    'Cantidad Inicial': item_data['initial_quantity'],
    'Cantidad Vendida': item_data['sold_quantity'],
    'Cantidad Disponible': item_data['available_quantity'],
    'Fecha de Creacion': date_created
})

In [9]:
# Utiliza json_normalize para crear un DataFrame directamente desde la lista de datos JSON
df = json_normalize(json_data_list)

In [10]:
df

,Marca,Modelo,Tipo producto,Condicion,Status,Precio,Moneda,Cantidad Inicial,Cantidad Vendida,Cantidad Disponible,Fecha de Creacion
0,Huawei,Y7,Cellphones,new,active,43999.99,ARS,3,1,1,18-07-2023
1,Huawei,P9 lite,Cellphones,new,active,32999.99,ARS,1,0,1,08-08-2023
2,Huawei,Y7 2019,Cellphones,new,active,38999.00,ARS,1,0,1,03-08-2023
3,Huawei,Rural,Cellphones,new,active,9999.00,ARS,25,0,1,04-07-2023
4,Huawei,Rural,Cellphones,new,active,9999.00,ARS,5,0,1,04-07-2023
...,...,...,...,...,...,...,...,...,...,...,...
145,Apple,iPhone SE (2nd Generation),Cellphones,new,active,659009.45,ARS,4,0,1,01-06-2023
146,Apple,Watch SE (GPS),Smartwatches,new,active,238999.00,ARS,4,1,1,25-05-2023
147,Apple,iPhone 13,Cellphones,new,active,1470099.21,ARS,1,0,1,03-07-2023
148,Apple,M1 2020,Notebooks,new,active,818207.00,ARS,2,0,1,08-08-2023


In [11]:
# Guarda el dataframe en un archivo plano delimitado por comas
df.to_csv('productos.csv', index=False, encoding='utf-8')